# Data Processsing
This notebook takes experimental data and converts it to a form that is usable for machine learning algorithm training.
This involves smoothing the data using the Savitzky–Golay filter, and adding slope values by comparing consecutive concentration values, and dividing by time elapsed. 


### Setup
Import neccessary packages and set current directory to the directory that contains this notebook

In [4]:
import pandas as pd
import numpy as np
from scipy.signal import savgol_filter 
import os
home_dir = os.getcwd().split('/notebooks')[0]
home_dir

'/scratch/garrettroell/machine_learning_clostridium/slope_prediction_lib'

### Import experimental data

In [5]:
exp_data = pd.read_csv(f'{home_dir}/experimental_data/experimental_data.csv')
exp_data.set_index(['composition','trial','time'],drop=True,inplace=True)

### Define and apply function to remove the first 24 hrs data
This is done to remove the effects of glucose feeding that take place in the first 24 hrs. All data points are shifted down by one day, and data points from the first day are removed

In [6]:
def subtract_one_day(df):
    df_copy = df.copy()
    df_copy.reset_index(inplace=True)
    df_copy = df_copy[df_copy['time'] > 1]
    df_copy['time'] = df_copy['time'] - 1
    df_copy.set_index(['composition', 'trial', 'time'], inplace=True, drop = True)
    return df_copy

In [7]:
raw_data = subtract_one_day(exp_data)
raw_data

acetate  biomass  butanol  butyrate  ethanol  \
composition trial time                                                  
1           1     0.580   21.610     0.41    0.040     0.060   10.940   
                  0.650   44.310     0.39    0.050     0.080   15.890   
                  1.020   46.190     0.46    0.210     0.640    8.140   
                  1.670   46.160     0.49    1.180     3.640   10.810   
                  3.700   34.390     0.64    8.440     9.760   20.340   
...                          ...      ...      ...       ...      ...   
10          2     2.403   58.616     1.32    0.944     6.015    3.473   
                  2.637   59.448     1.11    3.508    11.690   13.675   
                  3.643   65.943     1.44    2.159    10.895    8.682   
                  6.664   50.456     1.47    3.643    11.576   11.070   
                  7.570   48.125     1.51    4.857    12.371   13.675   

                         flow rate     H2   CO    CO2  
composition trial time                                 
1           1     0.580         20  0.125  0.5  0.375  
                  0.650         20  0.125  0.5  0.375  
                  1.020         20  0.125  0.5  0.375  
                  1.670         20  0.125  0.5  0.375  
                  3.700         20  0.125  0.5  0.375  
...                            ...    ...  ...    ...  
10          2     2.403          1  0.800  0.2  0.000  
                  2.637          1  0.800  0.2  0.000  
                  3.643          1  0.800  0.2  0.000  
                  6.664          1  0.800  0.2  0.000  
                  7.570          1  0.800  0.2  0.000  

[106 rows x 9 columns]

In [8]:
def smooth_interpolate(df,delta):
    """delta = step size"""
    savgol0 = lambda x: savgol_filter(x, 15, 2, deriv=0)
    smoothed_data = pd.DataFrame()
    
    composition = range (1,11)
    trial = [1,2]
    
    for comp in composition:
        for tri in trial:
            if (comp == 5 and tri == 2) or (comp == 6 and tri == 2) or (comp == 7 and tri == 2):
                pass
            else:
                section = df.loc[comp, tri]

                times = section.index
                max_time = times[-1] + delta
                new_times = np.arange(0,max_time, delta)  
                section = section.reindex(section.index.union(new_times))
                section = section.interpolate()
                section = section.round(3)
                times_to_remove = set(times) - (set(times) & set(new_times))
                section = section.loc[~section.index.isin(times_to_remove)]
            
                smooth=section.apply(savgol0)

                smooth.insert(0, "time", smooth.index)
                smooth.insert(0, "trial", tri)
                smooth.insert(0, "composition", comp)
                smoothed_data = pd.concat([smoothed_data , smooth],sort=True)


    smoothed_data.set_index(['composition','trial','time'],drop=True,inplace=True)
    smoothed_data.clip(lower=0,inplace=True)
        
    times_to_drop = np.arange(0,1,delta)
    smoothed_data = smoothed_data.drop(index =times_to_drop, level ='time')

    # work around to change index values
    smoothed_data.reset_index(drop=False,inplace=True)
    smoothed_data['time'] = round (smoothed_data['time']-1,1)
    smoothed_data.set_index(['composition','trial','time'],drop=True,inplace=True)
    
    return smoothed_data

In [9]:
smooth_data = smooth_interpolate(exp_data, 0.1)
smooth_data

CO    CO2     H2    acetate   biomass   butanol  \
composition trial time                                                     
1           1     0.0   0.5  0.375  0.125  16.905029  0.429546  0.029818   
                  0.1   0.5  0.375  0.125  18.001195  0.421076  0.028260   
                  0.2   0.5  0.375  0.125  19.831073  0.410803  0.028316   
                  0.3   0.5  0.375  0.125  22.196249  0.403602  0.031357   
                  0.4   0.5  0.375  0.125  25.293925  0.403709  0.028700   
...                     ...    ...    ...        ...       ...       ...   
10          2     7.2   0.2  0.000  0.800  48.933157  1.492815  4.343044   
                  7.3   0.2  0.000  0.800  48.716525  1.497124  4.472496   
                  7.4   0.2  0.000  0.800  48.533664  1.501679  4.607530   
                  7.5   0.2  0.000  0.800  48.384573  1.506481  4.748145   
                  7.6   0.2  0.000  0.800  48.269253  1.511529  4.894341   

                         butyrate    ethanol  flow rate  
composition trial time                                   
1           1     0.0    0.075529  15.895240       20.0  
                  0.1    0.061033  15.607023       20.0  
                  0.2    0.051013  15.144135       20.0  
                  0.3    0.050367  14.522308       20.0  
                  0.4    0.035514  13.956015       20.0  
...                           ...        ...        ...  
10          2     7.2   12.030206  12.560982        1.0  
                  7.3   12.115742  12.840664        1.0  
                  7.4   12.205961  13.135098        1.0  
                  7.5   12.300863  13.444285        1.0  
                  7.6   12.400447  13.768224        1.0  

[890 rows x 9 columns]

In [10]:
def add_slope_values(df):

  acetate_Δ = []
  biomass_Δ = []
  butanol_Δ = []
  butyrate_Δ = []
  ethanol_Δ = []

  old_comp = 0
  old_tri = 0

  for index, _ in df.iterrows():
    new_comp = index[0]
    new_tri = index[1]

    if (new_comp == old_comp and new_tri == old_tri):
      Δ_time = index[2] - old_index[2]

      acetate_Δ.append((df.loc[index]['acetate']-df.loc[old_index]['acetate'])/Δ_time)
      biomass_Δ.append((df.loc[index]['biomass']-df.loc[old_index]['biomass'])/Δ_time)
      butanol_Δ.append((df.loc[index]['butanol']-df.loc[old_index]['butanol'])/Δ_time)
      butyrate_Δ.append((df.loc[index]['butyrate']-df.loc[old_index]['butyrate'])/Δ_time)
      ethanol_Δ.append((df.loc[index]['ethanol']-df.loc[old_index]['ethanol'])/Δ_time)
    else: 
      acetate_Δ.append(0)
      biomass_Δ.append(0)
      butanol_Δ.append(0)
      butyrate_Δ.append(0)
      ethanol_Δ.append(0)

    old_comp = index[0]
    old_tri = index[1]
    old_index = index

  df['acetate_Δ'] = acetate_Δ
  df['biomass_Δ'] = biomass_Δ
  df['butanol_Δ'] = butanol_Δ
  df['butyrate_Δ'] = butyrate_Δ
  df['ethanol_Δ'] = ethanol_Δ

  return df

In [11]:
add_slope_values(smooth_data)

CO    CO2     H2    acetate   biomass   butanol  \
composition trial time                                                     
1           1     0.0   0.5  0.375  0.125  16.905029  0.429546  0.029818   
                  0.1   0.5  0.375  0.125  18.001195  0.421076  0.028260   
                  0.2   0.5  0.375  0.125  19.831073  0.410803  0.028316   
                  0.3   0.5  0.375  0.125  22.196249  0.403602  0.031357   
                  0.4   0.5  0.375  0.125  25.293925  0.403709  0.028700   
...                     ...    ...    ...        ...       ...       ...   
10          2     7.2   0.2  0.000  0.800  48.933157  1.492815  4.343044   
                  7.3   0.2  0.000  0.800  48.716525  1.497124  4.472496   
                  7.4   0.2  0.000  0.800  48.533664  1.501679  4.607530   
                  7.5   0.2  0.000  0.800  48.384573  1.506481  4.748145   
                  7.6   0.2  0.000  0.800  48.269253  1.511529  4.894341   

                         butyrate    ethanol  flow rate  acetate_Δ  biomass_Δ  \
composition trial time                                                          
1           1     0.0    0.075529  15.895240       20.0   0.000000   0.000000   
                  0.1    0.061033  15.607023       20.0  10.961665  -0.084697   
                  0.2    0.051013  15.144135       20.0  18.298778  -0.102733   
                  0.3    0.050367  14.522308       20.0  23.651756  -0.072009   
                  0.4    0.035514  13.956015       20.0  30.976760   0.001068   
...                           ...        ...        ...        ...        ...   
10          2     7.2   12.030206  12.560982        1.0  -2.504023   0.040625   
                  7.3   12.115742  12.840664        1.0  -2.166317   0.043090   
                  7.4   12.205961  13.135098        1.0  -1.828612   0.045554   
                  7.5   12.300863  13.444285        1.0  -1.490907   0.048019   
                  7.6   12.400447  13.768224        1.0  -1.153202   0.050483   

                        butanol_Δ  butyrate_Δ  ethanol_Δ  
composition trial time                                    
1           1     0.0    0.000000    0.000000   0.000000  
                  0.1   -0.015584   -0.144950  -2.882172  
                  0.2    0.000561   -0.100208  -4.628878  
                  0.3    0.030416   -0.006462  -6.218271  
                  0.4   -0.026579   -0.148525  -5.662923  
...                           ...         ...        ...  
10          2     7.2    1.238711    0.808534   2.649303  
                  7.3    1.294524    0.855361   2.796824  
                  7.4    1.350336    0.902189   2.944345  
                  7.5    1.406149    0.949017   3.091866  
                  7.6    1.461962    0.995845   3.239387  

[890 rows x 14 columns]

In [12]:
def add_previous_values(df):
    
    prev_acetate = []
    prev_biomass = []
    prev_butanol = []
    prev_butyrate = []
    prev_ethanol = []

    prev_acetate_Δ = []
    prev_biomass_Δ = []
    prev_butanol_Δ = []
    prev_butyrate_Δ = []
    prev_ethanol_Δ = []

    for count, (index, data) in enumerate(df.iterrows()):
        prev_data = df.iloc[count-1]
        time = index[2]
        
        if time == 0 or time == 0.1:
            prev_acetate.append(0)
            prev_biomass.append(0)
            prev_butanol.append(0)
            prev_butyrate.append(0)
            prev_ethanol.append(0)
    
            prev_acetate_Δ.append(0)
            prev_biomass_Δ.append(0)
            prev_butanol_Δ.append(0)
            prev_butyrate_Δ.append(0)
            prev_ethanol_Δ.append(0)
        else:
            prev_acetate.append(prev_data['acetate'])
            prev_biomass.append(prev_data['biomass'])
            prev_butanol.append(prev_data['butanol'])
            prev_butyrate.append(prev_data['butyrate'])
            prev_ethanol.append(prev_data['butanol'])
            
            prev_acetate_Δ.append(prev_data['acetate_Δ'])
            prev_biomass_Δ.append(prev_data['biomass_Δ'])
            prev_butanol_Δ.append(prev_data['butanol_Δ'])
            prev_butyrate_Δ.append(prev_data['butyrate_Δ'])
            prev_ethanol_Δ.append(prev_data['butyrate_Δ'])
            
    df['prev_acetate']  = prev_acetate
    df['prev_biomass']  = prev_biomass
    df['prev_butanol']  = prev_butanol
    df['prev_butyrate'] = prev_butyrate
    df['prev_ethanol']  = prev_ethanol

    df['prev_acetate_Δ']  = prev_acetate_Δ
    df['prev_biomass_Δ']  = prev_biomass_Δ
    df['prev_butanol_Δ']  = prev_butanol_Δ
    df['prev_butyrate_Δ'] = prev_butyrate_Δ
    df['prev_ethanol_Δ']  = prev_ethanol_Δ

    return df

In [13]:
smooth_data = add_previous_values(smooth_data)

In [14]:
# display(raw_data.head())
display(smooth_data.head())

CO    CO2     H2    acetate   biomass   butanol  \
composition trial time                                                     
1           1     0.0   0.5  0.375  0.125  16.905029  0.429546  0.029818   
                  0.1   0.5  0.375  0.125  18.001195  0.421076  0.028260   
                  0.2   0.5  0.375  0.125  19.831073  0.410803  0.028316   
                  0.3   0.5  0.375  0.125  22.196249  0.403602  0.031357   
                  0.4   0.5  0.375  0.125  25.293925  0.403709  0.028700   

                        butyrate    ethanol  flow rate  acetate_Δ  ...  \
composition trial time                                             ...   
1           1     0.0   0.075529  15.895240       20.0   0.000000  ...   
                  0.1   0.061033  15.607023       20.0  10.961665  ...   
                  0.2   0.051013  15.144135       20.0  18.298778  ...   
                  0.3   0.050367  14.522308       20.0  23.651756  ...   
                  0.4   0.035514  13.956015       20.0  30.976760  ...   

                        prev_acetate  prev_biomass  prev_butanol  \
composition trial time                                             
1           1     0.0       0.000000      0.000000      0.000000   
                  0.1       0.000000      0.000000      0.000000   
                  0.2      18.001195      0.421076      0.028260   
                  0.3      19.831073      0.410803      0.028316   
                  0.4      22.196249      0.403602      0.031357   

                        prev_butyrate  prev_ethanol  prev_acetate_Δ  \
composition trial time                                                
1           1     0.0        0.000000      0.000000        0.000000   
                  0.1        0.000000      0.000000        0.000000   
                  0.2        0.061033      0.028260       10.961665   
                  0.3        0.051013      0.028316       18.298778   
                  0.4        0.050367      0.031357       23.651756   

                        prev_biomass_Δ  prev_butanol_Δ  prev_butyrate_Δ  \
composition trial time                                                    
1           1     0.0         0.000000        0.000000         0.000000   
                  0.1         0.000000        0.000000         0.000000   
                  0.2        -0.084697       -0.015584        -0.144950   
                  0.3        -0.102733        0.000561        -0.100208   
                  0.4        -0.072009        0.030416        -0.006462   

                        prev_ethanol_Δ  
composition trial time                  
1           1     0.0         0.000000  
                  0.1         0.000000  
                  0.2        -0.144950  
                  0.3        -0.100208  
                  0.4        -0.006462  

[5 rows x 24 columns]

In [15]:
smooth_data.reset_index(inplace=True, drop = False)
smooth_data = smooth_data.loc[smooth_data['time'] != 0]
smooth_data = smooth_data.loc[smooth_data['time'] != 0.1]

smooth_data.set_index(['composition','trial','time'],drop=True,inplace=True)

In [16]:
smooth_data

CO    CO2     H2    acetate   biomass   butanol  \
composition trial time                                                     
1           1     0.2   0.5  0.375  0.125  19.831073  0.410803  0.028316   
                  0.3   0.5  0.375  0.125  22.196249  0.403602  0.031357   
                  0.4   0.5  0.375  0.125  25.293925  0.403709  0.028700   
                  0.5   0.5  0.375  0.125  29.005157  0.406214  0.031038   
                  0.6   0.5  0.375  0.125  32.899333  0.410848  0.043533   
...                     ...    ...    ...        ...       ...       ...   
10          2     7.2   0.2  0.000  0.800  48.933157  1.492815  4.343044   
                  7.3   0.2  0.000  0.800  48.716525  1.497124  4.472496   
                  7.4   0.2  0.000  0.800  48.533664  1.501679  4.607530   
                  7.5   0.2  0.000  0.800  48.384573  1.506481  4.748145   
                  7.6   0.2  0.000  0.800  48.269253  1.511529  4.894341   

                         butyrate    ethanol  flow rate  acetate_Δ  ...  \
composition trial time                                              ...   
1           1     0.2    0.051013  15.144135       20.0  18.298778  ...   
                  0.3    0.050367  14.522308       20.0  23.651756  ...   
                  0.4    0.035514  13.956015       20.0  30.976760  ...   
                  0.5    0.041123  13.322048       20.0  37.112317  ...   
                  0.6    0.081221  12.648214       20.0  38.941765  ...   
...                           ...        ...        ...        ...  ...   
10          2     7.2   12.030206  12.560982        1.0  -2.504023  ...   
                  7.3   12.115742  12.840664        1.0  -2.166317  ...   
                  7.4   12.205961  13.135098        1.0  -1.828612  ...   
                  7.5   12.300863  13.444285        1.0  -1.490907  ...   
                  7.6   12.400447  13.768224        1.0  -1.153202  ...   

                        prev_acetate  prev_biomass  prev_butanol  \
composition trial time                                             
1           1     0.2      18.001195      0.421076      0.028260   
                  0.3      19.831073      0.410803      0.028316   
                  0.4      22.196249      0.403602      0.031357   
                  0.5      25.293925      0.403709      0.028700   
                  0.6      29.005157      0.406214      0.031038   
...                              ...           ...           ...   
10          2     7.2      49.183559      1.488752      4.219173   
                  7.3      48.933157      1.492815      4.343044   
                  7.4      48.716525      1.497124      4.472496   
                  7.5      48.533664      1.501679      4.607530   
                  7.6      48.384573      1.506481      4.748145   

                        prev_butyrate  prev_ethanol  prev_acetate_Δ  \
composition trial time                                                
1           1     0.2        0.061033      0.028260       10.961665   
                  0.3        0.051013      0.028316       18.298778   
                  0.4        0.050367      0.031357       23.651756   
                  0.5        0.035514      0.028700       30.976760   
                  0.6        0.041123      0.031038       37.112317   
...                               ...           ...             ...   
10          2     7.2       11.949353      4.219173       -2.841728   
                  7.3       12.030206      4.343044       -2.504023   
                  7.4       12.115742      4.472496       -2.166317   
                  7.5       12.205961      4.607530       -1.828612   
                  7.6       12.300863      4.748145       -1.490907   

                        prev_biomass_Δ  prev_butanol_Δ  prev_butyrate_Δ  \
composition trial time                                                    
1           1     0.2        -0.084697       -0.015584        -0.144950   
                  0.3     

In [17]:
smooth_data.to_csv(f'{cur_dir}/processed_data/smooth_data.csv')